# unit 0.0 - Introduction to Python programing

Let us learn to program in python, but in a fun way!

We can start by programming a video-game: Pong. This is an old game that is like tennis or maybe better like hockey.

We will use [PyGame]( https://www.pygame.org/wiki/GettingStarted) to learn.

Let's dig in!

In [ ]:
import pygame # this is how you import a package in python
from pygame.locals import *

pygame.init() # initialize pygame
screen_size = (800,400)
screen = pygame.display.set_mode(screen_size,0,32) # create a screen
pygame.display.set_caption("Pong") # set the title of the window

# main loop of the game - repeating forever
running = True
while running:
    pygame.display.update() # update the screen to show the changes

    # all of this below is so when you press ESCAPE the game quits:
    for event in pygame.event.get():
        if event.type == QUIT:
            exit()
        elif event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                pygame.quit()
                exit()

This does not do much... displays a black screen. But there are no changes because we did not write any game code.

We will need to draw something. Let us start with a ball and two rackets.

In [1]:
import pygame
from pygame.locals import *

pygame.init()

screen_size = (800,400)
screen=pygame.display.set_mode(screen_size,0,32)
pygame.display.set_caption("Pong")

# create a surface to draw on
back = pygame.Surface(screen_size)
background = back.convert()
background.fill((0,0,0))

# create player paddles
paddle_size = 50
bar = pygame.Surface((10,paddle_size)) # create a surface (image) of size 10x50
# paddle 1:
paddle1 = bar.convert() # convert the surface to make blitting faster
paddle1.fill((0,0,255)) # fill the surface with blue color
# paddle 2:
paddle2 = bar.convert() # convert the surface to make blitting faster
paddle2.fill((255,0,0)) # fill the surface with red color

ball_sur = pygame.Surface((15,15)) # create a surface (image) of size 15x15
pygame.draw.circle(ball_sur,(0,255,0),(15/2,15/2),15/2) # draw a ball on the surface
ball = ball_sur.convert() # convert the surface to make blitting faster
ball.set_colorkey((0,0,0)) # make black color transparent

# set some initial positions:
paddle1_x, paddle2_x = 10., screen_size[0]-20. # x position of both bars - note the . after the number - this is needed because of the way python handles floating point numbers
paddle1_y, paddle2_y = screen_size[1]/2, screen_size[1]/2
ball_x, ball_y = screen_size[0]/2+25, screen_size[1]/2-15
paddle1_move, paddle2_move = 0. , 0.
# to animate our ball we need to change its position - lets define speed:
speed_x, speed_y, speed_ball = 250., 250., 250.
# displays the current score
player_score, computer_score = 0, 0
# define a clock for controlling the framerate
clock = pygame.time.Clock()
# font for displaying text (size 40)
font = pygame.font.SysFont("calibri", 40)

# main loop of the game - repeating forever
running = True
while running:
    # set screen
    screen.blit(background,(0,0))
    frame = pygame.draw.rect(screen,(255,255,255), Rect((5,5),
            (screen_size[0]-10, screen_size[1]-10)), 2)
    middle_line = pygame.draw.aaline(screen,(255,255,255),
            (screen_size[0]/2, 5), (screen_size[0]/2, screen_size[1]-5))
    
    # draw all game elements:
    paddle1_rect = screen.blit(paddle1,(paddle1_x, paddle1_y)) # draw the surface on the screen
    paddle2_rect = screen.blit(paddle2,(paddle2_x, paddle2_y)) # draw the surface on the screen
    ball_rect = screen.blit(ball,(ball_x, ball_y)) # draw the surface on the screen

    # scores:
    score1 = font.render(str(player_score), True,(255,255,255))
    score2 = font.render(str(computer_score), True,(255,255,255))
    screen.blit(score1,(screen_size[0]/2-50, screen_size[1]/2))
    screen.blit(score2,(screen_size[0]/2+50, screen_size[1]/2))

    paddle1_y += paddle1_move

    # ball movement:
    time_passed = clock.tick(60)
    time_sec = time_passed / 1000.0
    ball_x += speed_x * time_sec
    ball_y += speed_y * time_sec
    delta_move = speed_ball * time_sec

    # player2 movement (computer):
    if ball_x >= 305.:
        if not paddle2_y == ball_y + 7.5:
            if paddle2_y < ball_y + 7.5:
                paddle2_y += delta_move
            if  paddle2_y > ball_y - 42.5:
                paddle2_y -= delta_move
        else:
            paddle2_y == ball_y + 7.5
    
    # confine the paddle y movement to the screen:
    if paddle1_y >= screen_size[1]-paddle_size-10: paddle1_y = screen_size[1]-paddle_size-10
    elif paddle1_y <= 10. : paddle1_y = 10.
    if paddle2_y >= screen_size[1]-paddle_size-10: paddle2_y = screen_size[1]-paddle_size-10
    elif paddle2_y <= 10.: paddle2_y = 10.

    # confine the ball y movement to the screen:
    if ball_y <= 10.:
        speed_y = -speed_y
        ball_y = 10.
    elif ball_y >= screen_size[1]-22.5:
        speed_y = -speed_y
        ball_y = screen_size[1]-22.5

    # ball and paddles collisions:
    bp1_collision = ball_rect.colliderect(paddle1_rect)
    bp2_collision = ball_rect.colliderect(paddle2_rect)

    if bp1_collision == 1 or bp2_collision == 1:
        speed_x = -speed_x
        speed_y = -speed_y
        if bp1_collision == 1:
            ball_x = paddle1_x + 10.
        else:
            ball_x = paddle2_x - 10.

    # ball out of x bounds:
    if ball_x < 5.: # the player lost the round:
        computer_score += 1
        # reset the paddle and ball positions:
        paddle1_y, paddle2_y = screen_size[1]/2, screen_size[1]/2
        ball_x, ball_y = screen_size[0]/2+25, screen_size[1]/2-15
    elif ball_x > screen_size[0]-5: # the computer lost the round:
        player_score += 1
        # reset the paddle and ball positions:   
        paddle1_y, paddle2_y = screen_size[1]/2, screen_size[1]/2
        ball_x, ball_y = screen_size[0]/2+25, screen_size[1]/2-15
    
    
    # let us check and update for key-presses:
    for event in pygame.event.get():
        # this one checks if the user quits the program
        if event.type == QUIT:
            exit()
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                pygame.quit()
                exit()
            
            # this are players keys:
            if event.key == K_UP:
                paddle1_move = -delta_move
            elif event.key == K_DOWN:
                paddle1_move = delta_move
        
        elif event.type == KEYUP:
            if event.key == K_UP or event.key == K_DOWN:
                paddle1_move = 0.

    # update the screen - finally!
    pygame.display.update()

pygame 2.4.0 (SDL 2.26.4, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

Here is the full code - inspired by [this](https://www.pygame.org/project-Very+simple+Pong+game-816-.html)